## Lesson 03
### Classification. Logistic Regression.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# X[0] - ones (for intercept)
# X[1] - experience, years
# X[2] - cost per lesson
# X[3] - qualification rate
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [3]:
# Classes (target): 1 - teacher selected, 0 - teacher not selected
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

#### Standardization (Z-score Normalization)

In [4]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
# Create a copy array with standardized "Cost per Lesson" feature
X_std = X.copy()
X_std[2, :] = calc_std_feat(X[2, :])
X_std

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

#### Log Loss

In [6]:
# Classification metric based on probabilities
# Log Loss is -1 * the log of the Likelihood Function
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    # err = np.sum(err) # may be needed in some cases
    return err

In [7]:
# In this function, we get rid of '0' and '1' values in y_pred, to avoid zero in log
def calc_logloss_no_div_by_zero(y, y_pred):
    # Array pre-processing
    y_pred[y_pred == 1] = 1 - 1e-10
    y_pred[y_pred == 0] = 1e-10
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [8]:
# Test calc_logloss_no_div_by_zero function
y1 = np.array([0, 0, 1, 1], dtype = np.float64)
y_pred1 = np.array([0, 0.001, 0.999, 1], dtype = np.float64)
calc_logloss_no_div_by_zero(y1, y_pred1)

0.0005002502167917713

#### Sigmoid

$$\sigma(z) = \frac{1}{1 + exp^{(-z)}}$$
where $\sigma$ is `p` and $z=W*X$ ($W$ is Weight, $X$ is Feature)

In [9]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

#### Logistic Regression

In [37]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss_no_div_by_zero(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [79]:
W = eval_model(X_std, y, iterations=10000, alpha=1e-1)

1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734
2000 [-4.59094777 -1.0359998  -0.13825631  4.44501211] 0.35333924193451927
3000 [-5.91004767 -1.08773886 -0.62127728  5.36155604] 0.32501347614925347
4000 [-6.96133333 -1.14371437 -0.98694403  6.13057114] 0.3066241878369146
5000 [-7.85498033 -1.19935714 -1.28748713  6.80564285] 0.2931244675498327
6000 [-8.64731257 -1.25347799 -1.54786532  7.41682354] 0.28239334043313835
7000 [-9.36966717 -1.30586395 -1.78130777  7.98193089] 0.27340382430941684
8000 [-10.04075209  -1.35657036  -1.99546097   8.51208184] 0.26560192980297875
9000 [-10.67239108  -1.40571631  -2.1950503    9.01453862] 0.2586628760279488
10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.2523832489493557


In [67]:
def eval_model_reg_elasticnet(X, y, iterations, alpha=1e-4, lambda_l1_=1e-8, lambda_l2_=1e-10):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss_no_div_by_zero(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T) + (lambda_l1_ * np.sign(W)) + (lambda_l2_ * W))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [82]:
W = eval_model_reg_elasticnet(X_std, y, iterations=10000, alpha=1e-1, lambda_l1_=1e-2, lambda_l2_=1e-4)

1000 [-2.20383415 -0.83013077  0.49035947  2.67077698] 0.4305474747680781
2000 [-3.51923442e+00 -8.52532169e-01 -1.39899611e-04  3.49014675e+00] 0.38595313676167387
3000 [-4.07802626e+00 -9.64160196e-01  7.88603272e-05  4.00436439e+00] 0.3674484827051195
4000 [-4.36395515 -0.99930548 -0.05574578  4.23766313] 0.35967135919446663
5000 [-4.57065063 -1.00720795 -0.13034334  4.37885488] 0.35452213016765743
6000 [-4.72082334 -1.01259546 -0.18534305  4.48118894] 0.35091178101869447
7000 [-4.83104952 -1.01672226 -0.22549662  4.55676984] 0.34832564545211764
8000 [-4.91264044 -1.01988206 -0.25508142  4.61298935] 0.3464451532753515
9000 [-4.97341433 -1.02229328 -0.27704285  4.65501531] 0.3450627739711649
10000 [-5.01889298 -1.02412946 -0.29343586  4.68654761] 0.3440383921451049
